In [1]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 셀레니움 신버전 크롬 드라이버 패키지

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
# 진행률
from tqdm.notebook import tqdm
import re

In [9]:
def iframe_finder(iframe_id) :
    wait = WebDriverWait(driver, 10)
    driver.switch_to.default_content()
    # self.iframe
    iframe = wait.until(EC.presence_of_element_located((By.ID, iframe_id)))
    driver.switch_to.frame(iframe)
def is_action_successful(driver):
    try:
        wait = WebDriverWait(driver, 3)
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "N_KDL")))
        return True
    except:
        return False
# 1. 식당이름 입력
# 2. iframe searchIframe 변경
# 3. 페이지에 식당이 있는지 확인
# 4. 내가 찾으려는 식당 주소와 검색해서 나온 식당 주소 비교 
# 5. 찾았다면 클릭

# Chrome headless 설정
options = Options()
# options.add_argument('--headless')

# 드라이버 생성
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# name = '연화양꼬치'
# name = '맛깔참죽신제주점'
# name = '커피마마제주연동점'
name = '삼성혈해물탕'
# name = '순창갈비'
# name = '스톤아일랜드탭하우스'
# name = '진낙지앤'
name = '토크쇼코스모스점'
# 리뷰 페이지 열기
print(f'현재 크롤링이 진행중인 검색어 : {name}')

url = f"https://map.naver.com/v5/search/{name}"

driver.get(url)

wait = WebDriverWait(driver, 10)

iframe_finder('searchIframe')
# iframe_finder('entryIframe')

현재 크롤링이 진행중인 검색어 : 토크쇼코스모스점


In [10]:
iframe_finder('entryIframe')
dt = driver.find_elements(by = By.CLASS_NAME , value = 'zD5Nm.f7aZ0')

In [11]:
# 문제 수정 - 토그쇼 코스모스점

rest_img_test = driver.find_elements(by = By.CLASS_NAME , value ='K0PDV')

In [13]:
rest_img_test

[<selenium.webdriver.remote.webelement.WebElement (session="f14337d3063482e13b277f83a154d4a9", element="845C96E0BD3BCA3A9B40E8DA6D4669C3_element_56")>]

In [14]:
review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')

In [16]:
for i in review_information:
    print(i.text)

주소
제주 제주시 노연로 147
영업시간 정보 수정
이용시간을 알려주세요.
추가
가격 정보 수정 제안
가격표 사진을 올려주세요.
추가
사장님, 플레이스를
무료로 직접 관리하세요!
권한 받기


In [20]:
review_information[1].text

'영업시간 정보 수정\n이용시간을 알려주세요.\n추가'

In [23]:
driver.find_elements(by = By.CLASS_NAME , value = 'PIbes')[0].text

'주소\n제주 제주시 노연로 147\n영업시간 정보 수정\n이용시간을 알려주세요.\n추가\n가격 정보 수정 제안\n가격표 사진을 올려주세요.\n추가\n사장님, 플레이스를\n무료로 직접 관리하세요!\n권한 받기'

In [30]:
pattern1 = r'(접기|오늘 휴무|펼쳐보기|복사|내용 더보기|영업 중|안내\n복사|영업 종료|이용시간을 알려주세요.)'
pattern1_plus = r'(사장님, 플레이스를\n무료로 직접 관리하세요!\n권한 받기|영업시간 정보 수정\n|\n추가)'
pattern2 = r'\n(2,)'
pattern3 = r'\n\n'
n = 0
df2 = pd.DataFrame(columns = ['주소' , '찾아가는길', '영업시간' , '편의' , '설명' ,'전화번호' , '기타' , '이미지url'])

In [31]:
print('식당 정보를 가져오는 중입니다.')
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "PIbes")))
time.sleep(0.2)
review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
# 식당 정보 가져오기
for t in review_information :
    # print(t.text)
    # print(n)
    # 정규표현식으로 데이터 정리
    text = re.sub(pattern1, "", t.text)
    text = re.sub(pattern1_plus, "", text)
    text = re.sub(pattern2, "", text)
    text = re.sub(pattern3, "\n", text)
    text = text.rstrip('\n')
    # print(text)
    # print()
    n += 1
    if '주소' in text :
        text = re.sub('주소\n', "", text)
        df2['주소'] = [text]
    elif '찾아가는길' in text :
        text = re.sub('찾아가는길\n', "", text)
        df2['찾아가는길'] = [text]
    elif '영업시간' in text :
        text = re.sub('영업시간\n', "", text)
        df2['영업시간'] = [text]
    elif '064' in text :
        text = re.sub('전화번호\n', "", text)
        df2['전화번호'] = [text]
    elif '050' in text :
        text = re.sub('전화번호\n', "", text)
        df2['전화번호'] = [text]
    elif '편의' in text :
        text = re.sub('편의\n', "", text)
        df2['편의'] = [text]
    elif '설명' in text :
        text = re.sub('설명\n', "", text)
        df2['설명'] = [text]
    else :
        df2['기타'] = [text]
    # lst.append(text)

식당 정보를 가져오는 중입니다.


In [35]:
df2.기타.loc[0]

''

In [4]:
iframe_finder('entryIframe')
show_rest_info_click = driver.find_elements(by = By.CLASS_NAME , value ='rvCSr')

In [62]:
# style 속성 값 가져오기
style = element['style']

# URL 추출
url = style.split('src=')[1].split('&quot;')[1]

print(url)

KeyError: 'style'

In [47]:
rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')

In [69]:
rest_img

[<selenium.webdriver.remote.webelement.WebElement (session="4c74f5d9d223613fb523a0a507f19f95", element="A6E0CB258631C5BFA864426FC385B883_element_99")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c74f5d9d223613fb523a0a507f19f95", element="A6E0CB258631C5BFA864426FC385B883_element_98")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c74f5d9d223613fb523a0a507f19f95", element="A6E0CB258631C5BFA864426FC385B883_element_100")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c74f5d9d223613fb523a0a507f19f95", element="A6E0CB258631C5BFA864426FC385B883_element_101")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4c74f5d9d223613fb523a0a507f19f95", element="A6E0CB258631C5BFA864426FC385B883_element_102")>]

In [72]:
 url_test_data = rest_img[0].get_attribute('style')

In [102]:
# 이미지 url 가져오기

rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')
rest_img_urls = []
for stylejs in rest_img :
    
    url_test_data = stylejs.get_attribute('style')
    url_find_num = url_test_data.find('url("')
    if 'jpg' in url_test_data :
        url_find_num_end = url_test_data.find('jpg')
    else :
        url_find_num_end = url_test_data.find('.")')
    rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
    if rest_img_url != '' :
        rest_img_urls.append(rest_img_url)

In [103]:
rest_img_urls

['https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230325_247%2F1679719459070BBX5q_JPEG%2F20230324_184832.jpg',
 'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221031_49%2F16671891933914YOfP_JPEG%2F20221031_125312%25C1%25A4%25B8%25E9%25BB%25E7%25C1%25F8.jpg',
 'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzAzMTFfMTMz%2FMDAxNjc4NDY1MzkwMDA3.Lx3CirkXABoAaror2DUcqfGPyZZ-N8aS7r_VPSZN2MMg.bkWcsj4fZDFVt-YYJ8GXqp0BGMCuo2C5AAQp6IA2vVIg.JPEG.gogo89_%2FDSC00194.")',
 'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzAzMDRfMTky%2FMDAxNjc3OTA2NzM2OTA0.9bWEuxBQ_dAzCONyIABmrzng0sdgJsVFD4NTY8kCZvsg.5ST6SvAjBwC_bHRoSXsTKu9ISXHMUtOwzjo7LnLTZ_kg.JPEG.lamatem%2F10.")',
 'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=ht

In [113]:
# class rest_info () :
#     def __init__(self , rest_id) :
#         self.rest_id = rest_id
#         self.result_df = None
        
    

# 네이버 식당 정보 크롤링

In [36]:
# def rest_info(rest_name , rest_id) :    
    
#     name = rest_name
#     id_num = rest_id
    
#     # Chrome headless 설정
#     options = Options()
#     # options.add_argument('--headless')

#     # 드라이버 생성
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

#     # name = '고기다 성산흑돼지'
#     # 리뷰 페이지 열기
#     print(f'현재 크롤링이 진행중인 검색어 : {name}')

#     url = f"https://map.naver.com/v5/search/{name}/place/{id_num}"

#     driver.get(url)
#     wait = WebDriverWait(driver, 5)
#     iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
#     driver.switch_to.frame(iframe)
    
#     #iframe_finder('entryIframe')

#     try :
#         wait.until(EC.presence_of_element_located((By.CLASS_NAME, "rvCSr")))
#         time.sleep(0.2)
#         show_rest_info_click = driver.find_elements(by = By.CLASS_NAME , value ='rvCSr')
#         if show_rest_info_click is not None :
#             for click in show_rest_info_click :
#                 click.click()
#                 scarp_rest_info = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
#     except :
#         print('펼칠 내용이 없습니다.')
#         pass

#     # result = re.sub(pattern, "", text_2)
#     n = 0
#     # lst = []
#     df = pd.DataFrame(columns = ['주소' , '찾아가는길', '영업시간' , '편의' , '설명' ,'전화번호' , '기타' , '이미지url'])
#     pattern1 = r'(접기|오늘 휴무|펼쳐보기|복사|내용 더보기|영업 중|안내\n복사|영업 종료|이용시간을 알려주세요.)'
#     pattern1_plus = r'(사장님, 플레이스를\n무료로 직접 관리하세요!\n권한 받기|영업시간 정보 수정\n|\n추가)'
#     pattern2 = r'\n(2,)'
#     pattern3 = r'\n\n'
#     # 이미지 url 가져오기
#     try :
#         print('이미지 url 가져오는중')
#         wait.until(EC.presence_of_element_located((By.CLASS_NAME, "fNygA")))
#         time.sleep(0.3)

#         rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')
#         rest_img_urls = []
#         for stylejs in rest_img :

#             url_test_data = stylejs.get_attribute('style')
#             url_find_num = url_test_data.find('url("')
#             if 'jpg' in url_test_data :
#                 url_find_num_end = url_test_data.find('jpg')
#             else :
#                 url_find_num_end = url_test_data.find('.")')
#             rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
#             if rest_img_url != '' :
#                 rest_img_urls.append(rest_img_url)
#         df['이미지url'] = [rest_img_urls]
#     except : 
#         # 추가 식당 정보 이미지가 한 개일 경우 위 코드 복붙
#         try :
#             wait.until(EC.presence_of_element_located((By.CLASS_NAME, "K0PDV")))
#             time.sleep(0.3)



#             rest_img = driver.find_elements(by = By.CLASS_NAME , value ='K0PDV')
#             rest_img_urls = []
#             for stylejs in rest_img :

#                 url_test_data = stylejs.get_attribute('style')
#                 url_find_num = url_test_data.find('url("')
#                 if 'jpg' in url_test_data :
#                     url_find_num_end = url_test_data.find('jpg')
#                 else :
#                     url_find_num_end = url_test_data.find('.")')
#                 rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
#                 if rest_img_url != '' :
#                     rest_img_urls.append(rest_img_url)
#             df['이미지url'] = [rest_img_urls]
#         except : 
#             print('No restaurant image')
#             pass
    
    
#     print('이미지 url 가져오기 종료')
    
    
#     try :
#         print('식당 정보를 가져오는 중입니다.')
#         wait.until(EC.presence_of_element_located((By.CLASS_NAME, "PIbes")))
#         time.sleep(0.2)
#         review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
#         # 식당 정보 가져오기
#         for t in review_information :
#             # print(t.text)
#             # print(n)
#             # 정규표현식으로 데이터 정리
#             text = re.sub(pattern1, "", t.text)
#             text = re.sub(pattern1_plus, "", text)
#             text = re.sub(pattern2, "", text)
#             text = re.sub(pattern3, "\n", text)
#             text = text.rstrip('\n')
#             # print(text)
#             # print()
#             n += 1
#             if text != '' :
#                 if '주소' in text :
#                     text = re.sub('주소\n', "", text)
#                     df['주소'] = [text]
#                 elif '찾아가는길' in text :
#                     text = re.sub('찾아가는길\n', "", text)
#                     df['찾아가는길'] = [text]
#                 elif '영업시간' in text :
#                     text = re.sub('영업시간\n', "", text)
#                     df['영업시간'] = [text]
#                 elif '064' in text :
#                     text = re.sub('전화번호\n', "", text)
#                     df['전화번호'] = [text]
#                 elif '050' in text :
#                     text = re.sub('전화번호\n', "", text)
#                     df['전화번호'] = [text]
#                 elif '편의' in text :
#                     text = re.sub('편의\n', "", text)
#                     df['편의'] = [text]
#                 elif '설명' in text :
#                     text = re.sub('설명\n', "", text)
#                     df['설명'] = [text]
#                 else :
#                     df['기타'] = [text]
#             else : 
#                 pass
#             # lst.append(text)
#     except :
#         print('식당 정보를 가져오지 못했습니다.')
#         pass
#     print('식당 정보 가져오기 종료')
#     driver.quit()
#     print('크롤링 종료')
#     print()
    
#     return df

In [ ]:
def rest_info(rest_name , rest_id , address) :    
    # address 는 제주시연동 과 같이 붙여서 작성할 것
    
    name = rest_name
    id_num = rest_id
    
    # Chrome headless 설정
    options = Options()
    # options.add_argument('--headless')

    # 드라이버 생성
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # name = '고기다 성산흑돼지'
    # 리뷰 페이지 열기
    print(f'현재 크롤링이 진행중인 검색어 : {name}')

    url = f"https://map.naver.com/v5/search/{name}/place/{id_num}"

    driver.get(url)
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
    driver.switch_to.frame(iframe)
    
    #iframe_finder('entryIframe')

    try :
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "rvCSr")))
        time.sleep(0.2)
        show_rest_info_click = driver.find_elements(by = By.CLASS_NAME , value ='rvCSr')
        if show_rest_info_click is not None :
            for click in show_rest_info_click :
                click.click()
                scarp_rest_info = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
    except :
        print('펼칠 내용이 없습니다.')
        pass

    # result = re.sub(pattern, "", text_2)
    n = 0
    # lst = []
    df = pd.DataFrame(columns = ['주소' , '찾아가는길', '영업시간' , '편의' , '설명' ,'전화번호' , '기타' , '이미지url'])
    pattern1 = r'(접기|오늘 휴무|펼쳐보기|복사|내용 더보기|영업 중|안내\n복사|영업 종료|이용시간을 알려주세요.)'
    pattern1_plus = r'(사장님, 플레이스를\n무료로 직접 관리하세요!\n권한 받기|영업시간 정보 수정\n|\n추가)'
    pattern2 = r'\n(2,)'
    pattern3 = r'\n\n'
    # 이미지 url 가져오기
    try :
        print('이미지 url 가져오는중')
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "fNygA")))
        time.sleep(0.3)

        rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')
        rest_img_urls = []
        for stylejs in rest_img :

            url_test_data = stylejs.get_attribute('style')
            url_find_num = url_test_data.find('url("')
            if 'jpg' in url_test_data :
                url_find_num_end = url_test_data.find('jpg')
            else :
                url_find_num_end = url_test_data.find('.")')
            rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
            if rest_img_url != '' :
                rest_img_urls.append(rest_img_url)
        df['이미지url'] = [rest_img_urls]
    except : 
        # 추가 식당 정보 이미지가 한 개일 경우 위 코드 복붙
        try :
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "K0PDV")))
            time.sleep(0.3)



            rest_img = driver.find_elements(by = By.CLASS_NAME , value ='K0PDV')
            rest_img_urls = []
            for stylejs in rest_img :

                url_test_data = stylejs.get_attribute('style')
                url_find_num = url_test_data.find('url("')
                if 'jpg' in url_test_data :
                    url_find_num_end = url_test_data.find('jpg')
                else :
                    url_find_num_end = url_test_data.find('.")')
                rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
                if rest_img_url != '' :
                    rest_img_urls.append(rest_img_url)
            df['이미지url'] = [rest_img_urls]
        except : 
            print('No restaurant image')
            pass
    
    
    print('이미지 url 가져오기 종료')
    
    
    try :
        print('식당 정보를 가져오는 중입니다.')
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "PIbes")))
        time.sleep(0.2)
        review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
        # 식당 정보 가져오기
        for t in review_information :
            # print(t.text)
            # print(n)
            # 정규표현식으로 데이터 정리
            text = re.sub(pattern1, "", t.text)
            text = re.sub(pattern1_plus, "", text)
            text = re.sub(pattern2, "", text)
            text = re.sub(pattern3, "\n", text)
            text = text.rstrip('\n')
            # print(text)
            # print()
            n += 1
            if text != '' :
                if '주소' in text :
                    text = re.sub('주소\n', "", text)
                    df['주소'] = [text]
                elif '찾아가는길' in text :
                    text = re.sub('찾아가는길\n', "", text)
                    df['찾아가는길'] = [text]
                elif '영업시간' in text :
                    text = re.sub('영업시간\n', "", text)
                    df['영업시간'] = [text]
                elif '064' in text :
                    text = re.sub('전화번호\n', "", text)
                    df['전화번호'] = [text]
                elif '050' in text :
                    text = re.sub('전화번호\n', "", text)
                    df['전화번호'] = [text]
                elif '편의' in text :
                    text = re.sub('편의\n', "", text)
                    df['편의'] = [text]
                elif '설명' in text :
                    text = re.sub('설명\n', "", text)
                    df['설명'] = [text]
                else :
                    df['기타'] = [text]
            else : 
                pass
            # lst.append(text)
    except :
        print('식당 정보를 가져오지 못했습니다.')
        pass
    print('식당 정보 가져오기 종료')
    driver.quit()
    print('크롤링 종료')
    print()
    df['주소id'] = address
    data['사업장명'] = rest_name
    data['id'] = rest_id
    return df

In [3]:
연동_data_df=pd.read_csv('제주도식당/제주시/연동/id/연동_id_name.csv' , encoding= 'euc-kr')
# 연동_data_df.drop(연동_data_df.columns[0] , axis = 1 , inplace = True)

In [40]:
연동_data_df.info

<bound method DataFrame.info of               사업장명          id
0            봉플라봉뱅  1713847517
1       스톤아일랜드탭하우스  1244422132
2            연화양꼬치  1482063535
3       24시누름돌김치찌개  1028483996
4              영화루  1109507832
...            ...         ...
1014      토라네코제주본점  1778945379
1015    지에스25연동가락점  1326118478
1016   귀아랑2길베이커리카페  1887309829
1017  세븐일레븐제주연동노연점  1255085555
1018      카페노연로117  1971746359

[1019 rows x 2 columns]>

In [39]:
import os

directory_path = '제주도식당/제주시/연동/식당정보'  # 디렉토리 경로

if not os.path.exists(directory_path):  # 디렉토리가 존재하지 않으면
    os.makedirs(directory_path)  # 디렉토리 생성
    print('디렉토리가 생성되었습니다.')
else:
    print('디렉토리가 이미 존재합니다.')


print()
count = 1
check_lst = []

for name , rest_id in zip(연동_data_df.사업장명.tolist()[42:] , 연동_data_df.id.tolist()[42:]) :
    print(count)
    crawling_rest_info = rest_info(name , rest_id)
    check_lst.append(crawling_rest_info)
    crawling_rest_info.to_excel(f'제주도식당/제주시/연동/식당정보/{name}_info.xlsx' , index = False , encoding = 'euc-kr')
    count += 1

디렉토리가 이미 존재합니다.

1
현재 크롤링이 진행중인 검색어 : 해빈촌
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

2
현재 크롤링이 진행중인 검색어 : 맛동산
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

3
현재 크롤링이 진행중인 검색어 : 파도국수
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

4
현재 크롤링이 진행중인 검색어 : 평화가든
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

5
현재 크롤링이 진행중인 검색어 : 제원삼계탕
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

6
현재 크롤링이 진행중인 검색어 : 해피소울라운지
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

7
현재 크롤링이 진행중인 검색어 : 고슬
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

8
현재 크롤링이 진행중인 검색어 : 연동대복관
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

9
현재 크롤링이 진행중인 검색어 : 제주영재숯불
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

10
현재 크롤링이 진행중인 검색어 : 디케이

펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

78
현재 크롤링이 진행중인 검색어 : 슈퍼보틀
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

79
현재 크롤링이 진행중인 검색어 : 크라운호프연동점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

80
현재 크롤링이 진행중인 검색어 : 내가좋아하는것들
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

81
현재 크롤링이 진행중인 검색어 : 화정연화
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

82
현재 크롤링이 진행중인 검색어 : 낭푼밥상
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

83
현재 크롤링이 진행중인 검색어 : 거북이
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

84
현재 크롤링이 진행중인 검색어 : 갓포아키제주도점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

85
현재 크롤링이 진행중인 검색어 : 철수네숯불갈비
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

86
현재 크롤링이 진행중인 검색어 : 원조부안집연동점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입

현재 크롤링이 진행중인 검색어 : 통큰장어신제주점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

155
현재 크롤링이 진행중인 검색어 : 대박왕갈비연동점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

156
현재 크롤링이 진행중인 검색어 : 만운성
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

157
현재 크롤링이 진행중인 검색어 : 웰빙밥도둑
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

158
현재 크롤링이 진행중인 검색어 : 신제주삼보
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

159
현재 크롤링이 진행중인 검색어 : 문성반점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

160
현재 크롤링이 진행중인 검색어 : 번개포차본점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

161
현재 크롤링이 진행중인 검색어 : 산전수전인생역전
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

162
현재 크롤링이 진행중인 검색어 : 꼬방
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

163
현

이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

230
현재 크롤링이 진행중인 검색어 : 스페샬삼무점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

231
현재 크롤링이 진행중인 검색어 : 식객의주방
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

232
현재 크롤링이 진행중인 검색어 : 컴포즈커피제주제원점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

233
현재 크롤링이 진행중인 검색어 : 한양춘향연화점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

234
현재 크롤링이 진행중인 검색어 : 낙지로
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

235
현재 크롤링이 진행중인 검색어 : 복덕이네
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

236
현재 크롤링이 진행중인 검색어 : 버닝썬
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

237
현재 크롤링이 진행중인 검색어 : 호식이두마리치킨연동신시가지점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

238
현재 크롤링이 진행중인 검색어 : 흥부네정식
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:01<00:00, 5.98MB/s]


...

현재 크롤링이 진행중인 검색어 : 오늘김밥
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

628
현재 크롤링이 진행중인 검색어 : 제주가
펼칠 내용이 없습니다.
이미지 url 가져오는중
No restaurant image
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보를 가져오지 못했습니다.
식당 정보 가져오기 종료
크롤링 종료

629
현재 크롤링이 진행중인 검색어 : 다올레횟집
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

630
현재 크롤링이 진행중인 검색어 : 영갈분식
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

631
현재 크롤링이 진행중인 검색어 : 조선화로집제주연북로점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

632
현재 크롤링이 진행중인 검색어 : 민들레소반
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

633
현재 크롤링이 진행중인 검색어 : 순수한둠비연동점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

634
현재 크롤링이 진행중인 검색어 : 모네의화원
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

635
현재 크롤링이 진행중인 검색어 : 온도70
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료


식당 정보 가져오기 종료
크롤링 종료

703
현재 크롤링이 진행중인 검색어 : 차달임
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

704
현재 크롤링이 진행중인 검색어 : 충무로38양곱창제주점
펼칠 내용이 없습니다.
이미지 url 가져오는중
No restaurant image
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보를 가져오지 못했습니다.
식당 정보 가져오기 종료
크롤링 종료

705
현재 크롤링이 진행중인 검색어 : 우리솥해장국
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

706
현재 크롤링이 진행중인 검색어 : 제원칼집별관
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

707
현재 크롤링이 진행중인 검색어 : 엠박스제주
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

708
현재 크롤링이 진행중인 검색어 : 카페시도
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

709
현재 크롤링이 진행중인 검색어 : 제주연탄길
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

710
현재 크롤링이 진행중인 검색어 : 포비게러지
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

711
현재 크롤링이 진행중인 검색어 : 플래닛제주
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료

식당 정보 가져오기 종료
크롤링 종료

779
현재 크롤링이 진행중인 검색어 : 인베이드피씨카페
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

780
현재 크롤링이 진행중인 검색어 : 지에스25연동모디움점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

781
현재 크롤링이 진행중인 검색어 : 세븐일레븐연동국기점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

782
현재 크롤링이 진행중인 검색어 : 공차제주연동신시가지점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

783
현재 크롤링이 진행중인 검색어 : 공차제주연동점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

784
현재 크롤링이 진행중인 검색어 : 홍요깃거리애월점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

785
현재 크롤링이 진행중인 검색어 : 길용당
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

786
현재 크롤링이 진행중인 검색어 : 에이바우트커피신라신성점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

787
현재 크롤링이 진행중인 검색어 : 까미노
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오

현재 크롤링이 진행중인 검색어 : 리치망고신제주점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

854
현재 크롤링이 진행중인 검색어 : 심지유향
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

855
현재 크롤링이 진행중인 검색어 : 아이갓에브리씽
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

856
현재 크롤링이 진행중인 검색어 : 화목원
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

857
현재 크롤링이 진행중인 검색어 : 지에스25크라운점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

858
현재 크롤링이 진행중인 검색어 : 춤추는염소들
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

859
현재 크롤링이 진행중인 검색어 : 마농떡볶이
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

860
현재 크롤링이 진행중인 검색어 : 하나맘김밥
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

861
현재 크롤링이 진행중인 검색어 : 메가엠지씨커피제주연동점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

862
현재 크롤링이 진행중인 검색어 : (주)지앤엘
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지

이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

928
현재 크롤링이 진행중인 검색어 : 커피핌
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

929
현재 크롤링이 진행중인 검색어 : 아임누
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

930
현재 크롤링이 진행중인 검색어 : 디오커피랩
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

931
현재 크롤링이 진행중인 검색어 : 세븐일레븐제주현대오피스텔점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

932
현재 크롤링이 진행중인 검색어 : 봄봄신제주점
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

933
현재 크롤링이 진행중인 검색어 : 세븐일레븐제주연동제일점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

934
현재 크롤링이 진행중인 검색어 : 헬시바비
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

935
현재 크롤링이 진행중인 검색어 : 씨유제주스카이점
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

936
현재 크롤링이 진행중인 검색어 : 이룸갤러리카페
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종

In [120]:
test_df = 연동_data_df[:5]

In [123]:
test_df

,사업장명,id
0,봉플라봉뱅,1713847517
1,스톤아일랜드탭하우스,1244422132
2,연화양꼬치,1482063535
3,24시누름돌김치찌개,1028483996
4,영화루,1109507832


In [125]:
test_df.사업장명.tolist()

['봉플라봉뱅', '스톤아일랜드탭하우스', '연화양꼬치', '24시누름돌김치찌개', '영화루']

In [126]:
test_df.id.tolist()

[1713847517, 1244422132, 1482063535, 1028483996, 1109507832]

In [154]:
test_lst = []
count = 1
for name , rest_id in zip(test_df.사업장명.tolist() , test_df.id.tolist()) :
    print(count)
    test_lst.append(rest_info(name , rest_id))
    count += 1

1
현재 크롤링이 진행중인 검색어 : 봉플라봉뱅
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

2
현재 크롤링이 진행중인 검색어 : 스톤아일랜드탭하우스
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

3
현재 크롤링이 진행중인 검색어 : 연화양꼬치
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

4
현재 크롤링이 진행중인 검색어 : 24시누름돌김치찌개
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료

5
현재 크롤링이 진행중인 검색어 : 영화루
펼칠 내용이 없습니다.
이미지 url 가져오는중
이미지 url 가져오기 종료
식당 정보를 가져오는 중입니다.
식당 정보 가져오기 종료
크롤링 종료



In [156]:
test_lst[0]

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 문송1길 6-1 1층 봉플라봉뱅,제주도청 주차장 뒷골목,22:00에 라스트오더\n22시 0분에 라스트오더,"단체석, 주차, 예약, 무선 인터넷","Lunch 12:00 ~ 14:30 (목, 금, 토)\nDinner 17:30 ~ ...",0507-1408-0417,정보 수정 제안\n정보 수정 제안하기,[https://search.pstatic.net/common/?autoRotate...


In [157]:
test_lst[1]

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 삼무로7길 16 1층 스톤아일랜드탭하우스,NaN,23:00에 라스트오더\n23시 0분에 라스트오더,"단체석, 주차, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분",저희 스톤아일랜드탭하우스는 제주최초로 셀프시스템을 도입한 스마트펍으로 20가지 수제...,0507-1410-9009,홈페이지\nhttp://instagram.com/stoneisland_taphouse,[https://search.pstatic.net/common/?autoRotate...


In [158]:
test_lst[2]

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 연동8길 28,NaN,영업시간 정보 수정\n이용시간을 알려주세요.\n추가,NaN,1년미만 부드러운 양고기를 준비하고 있습니다,064-749-1001,NaN,[https://search.pstatic.net/common/?autoRotate...


In [159]:
test_lst[3]

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 노연로 146,NaN,24:00에 영업 종료\n0시 0분에 영업 종료,NaN,NaN,064-742-7076,수상 및 인증\n농림축산식품부 제공\n안심식당\n안심식당은 '덜어먹기 가능한 도구 ...,[https://search.pstatic.net/common/?autoRotate...


In [160]:
test_lst[4]

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 사장길 15-1,NaN,21:00에 영업 종료\n21시 0분에 영업 종료,NaN,NaN,064-742-8285,NaN,[https://search.pstatic.net/common/?autoRotate...


In [165]:
print(test_lst[4].이미지url.loc[0][1])

https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA1MjBfMjgz%2FMDAxNjg0NTgxOTAxMTE3.USry_K1J7eo9jtdAo5Ff2wRdyNn5cF3p4kQtDz7ErrUg.9Kmpj3unyZF960vdJKnS6a4pwQOWlJ0OXf8mLmcLfVwg.JPEG.mahoshi%2F1684581547922.jpg


# 내부 코드

In [110]:
#iframe_finder('entryIframe')

try :
    show_rest_info_click = driver.find_elements(by = By.CLASS_NAME , value ='rvCSr')
    if show_rest_info_click is not None :
        for click in show_rest_info_click :
            click.click()
            scarp_rest_info = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
except :
    print('펼칠 내용이 없습니다.')
    pass

# result = re.sub(pattern, "", text_2)
n = 0
# lst = []
df = pd.DataFrame(columns = ['주소' , '찾아가는길', '영업시간' , '편의' , '설명' ,'전화번호' , '기타' , '이미지url'])
        
# 이미지 url 가져오기


try :

    pattern1 = r'(접기|오늘 휴무|펼쳐보기|복사|내용 더보기|영업 중|안내\n복사)'
    pattern2 = r'\n(2,)'
    pattern3 = r'\n\n'

    rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')
    rest_img_urls = []
    for stylejs in rest_img :

        url_test_data = stylejs.get_attribute('style')
        url_find_num = url_test_data.find('url("')
        if 'jpg' in url_test_data :
            url_find_num_end = url_test_data.find('jpg')
        else :
            url_find_num_end = url_test_data.find('.")')
        rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
        rest_img_urls.append(rest_img_url)
    df['이미지url'] = [rest_img_urls]

    review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
except : 
    print('No restaurant image')
    pass

try :
    # 식당 정보 가져오기
    for t in review_information :
        # print(t.text)
        # print(n)
        # 정규표현식으로 데이터 정리
        text = re.sub(pattern1, "", t.text)
        text = re.sub(pattern2, "", text)
        text = re.sub(pattern3, "\n", text)
        text = text.rstrip('\n')
        # print(text)
        # print()
        n += 1
        if '주소' in text :
            text = re.sub('주소\n', "", text)
            df['주소'] = [text]
        elif '찾아가는길' in text :
            text = re.sub('찾아가는길\n', "", text)
            df['찾아가는길'] = [text]
        elif '영업시간' in text :
            text = re.sub('영업시간\n', "", text)
            df['영업시간'] = [text]
        elif '064' in text :
            text = re.sub('전화번호\n', "", text)
            df['전화번호'] = [text]
        elif '050' in text :
            text = re.sub('전화번호\n', "", text)
            df['전화번호'] = [text]
        elif '편의' in text :
            text = re.sub('편의\n', "", text)
            df['편의'] = [text]
        elif '설명' in text :
            text = re.sub('설명\n', "", text)
            df['설명'] = [text]
        else :
            df['기타'] = [text]
        # lst.append(text)
except :
    print('식당 정보를 가져오지 못했습니다.')
    pass
        

In [111]:
df

,주소,찾아가는길,영업시간,편의,설명,전화번호,기타,이미지url
0,제주 제주시 선덕로5길 20 삼성혈 해물탕 본점,제주도청 바로 뒤편에 위치하며 주차는\n본 업소 바로 앞 공영주차장 (30분 무료....,19:50에 라스트오더\n19시 50분에 라스트오더,"포장, 예약, 무선 인터넷, 남/녀 화장실 구분",원조 삼성혈 해물탕 본점은 약20년동안 오로지 해물탕만을 고집하는 원조맛집으로서 살...,0507-1421-3002,NaN,[https://search.pstatic.net/common/?autoRotate...


In [ ]:
# 추가로 이미지도 가져와보자.
# 주의 아래에는 파일 경로를 지정해주어야한다! 
# 윈도우에서 \로 경로를 지정하는데 \ 사용시 컴퓨터가 경로를 못 읽으므로 / 사용을 권장한다.

# url img를 내려받을 수 있는 패키지
from urllib.request import urlopen , urlretrieve
from urllib.error   import HTTPError
from urllib.error   import URLError


import os
# 폴더 생성
img_folder = 'C:/Users/PiGiraffe0/Downloads/test'


# 특정 폴더를 생성합니다.

if not os.path.isdir(img_folder) :
    os.mkdir(img_folder)

# 이미지 경로를 통해 이미지 파일에 이미지 저장하기.

for idx, link in enumerate(img_lst) :
    #.get_attribute('src') 은 src에 적혀있는 이미지 주소를 뽑아준다
    print(idx , '-' , link.get_attribute('src'))
    # urlretrieve 입력받은 이미지 url에 따라 이미지를 다운받고 특정 경로에 저장한다. 
    # f'' 형식은 포메팅이다 f'{}' {변수 : 변수 값을 반환한다.} 
    urlretrieve(link.get_attribute('src') , f'C:/Users/PiGiraffe0/Downloads/test/test_{idx}.jpg')
